In [3]:
import requests
import time
import pickle
import os
from bs4 import BeautifulSoup

In [4]:
# create list of index page links to later download
# example links
link = "https://www.theonion.com/opinion/american-voices"
link2 = "https://www.theonion.com/opinion/american-voices?startIndex=20"

index_links = []

article_n = 6500 # amount of articles
articles_per_page = 20
index_n = int(article_n/articles_per_page) # amount of index pages

for i in range(index_n):
    index_links.append(link + "?startIndex=" + str(i * articles_per_page))

print(len(index_links))
print(index_links[1])

325
https://www.theonion.com/opinion/american-voices?startIndex=20


In [6]:
index_pages_file = "index_pages.txt"
index_pages = [] # HTML content of index pages
 
if os.path.isfile(index_pages_file): 
    with open(index_pages_file, 'rb') as f:
        content = f.read()
        index_pages = pickle.loads(content)
        print("loaded pickled file")
        print(len(index_pages))

In [7]:
try:
    for i in range(len(index_pages), len(index_links)):
        print(i)
        index_link = index_links[i]
        index_pages.append(requests.get(index_link).text) 
except:
    pass
serialized = pickle.dumps(index_pages) 
with open(index_pages_file, 'wb') as f:
    f.write(serialized)

0
1
2
3
4
5


In [8]:
article_links = []

for i in range(len(index_pages)):
    html = index_pages[i]
    soup = BeautifulSoup(html, 'html.parser')
    soup_a =  soup.find_all('article')
    for i in soup_a:
        article_links.append( i.find('a', class_='sc-1out364-0 dPMosf js_link')['href'] )

len(article_links)

100

In [11]:
article_htmls_file = "article_htmls.txt"
article_htmls = [] # HTML content of index pages
 
if os.path.isfile(article_htmls_file): 
    with open(article_htmls_file, 'rb') as f:
        content = f.read()
        article_htmls = pickle.loads(content)
        print("loaded pickled file")

In [12]:
article_htmls_file = "article_htmls.txt"
try:
    for i in range(len(article_htmls), len(article_links)):
        link = article_links[i]
        article_htmls.append(requests.get(link).text) 
        print(i)
        #time.sleep(0.5)
except:
    pass
serialized = pickle.dumps(article_htmls) 
with open(article_htmls_file, 'wb') as f:
    f.write(serialized)

0
1
2
3
4
5


In [13]:
article_texts_file = "article_texts.txt"
article_texts = [] # Filtered article contents
 
if os.path.isfile(article_texts_file): 
    with open(article_texts_file, 'rb') as f:
        content = f.read()
        article_texts = pickle.loads(content)
        print("loaded pickled file")
        print(len(article_texts))


loaded pickled file
6495


In [14]:
try: 
    for i in range(len(article_texts), len(article_texts)):
        try:
            html = article_htmls[i]
            soup = BeautifulSoup(html, 'html.parser')
            header =  soup.find('header', class_="sc-1efpnfq-1 blDizU").text
            subtitle = soup.find('p', class_="sc-77igqf-0 fnnahv").text

            comments = soup.find_all('div', class_="sc-q35npn-1 dGOWqB") 
            comments_text = [ x.find('p', class_="sc-q35npn-3 cKacox").text for x in comments ]
            comments_names = [ x.find('p', class_="sc-q35npn-4 bfrXyU").text for x in comments ]

            comments_zipped = list(zip(comments_text, comments_names))

            article_texts.append([header, subtitle, comments_zipped])
        except AttributeError as err: 
            print(f"Attribute Error at {i}")
except KeyboardInterrupt as err:
    pass
serialized = pickle.dumps(article_texts) 
with open(article_texts_file, 'wb') as f:
    f.write(serialized)

In [16]:
print(len(article_texts))
print(article_texts[0])
print(article_texts[20])
print(article_texts[100])
print(article_texts[1000])
print(article_texts[2000])
print(article_texts[3000])
print(article_texts[4000])
#for i in range(30):
    #print(article_texts[i])
# print(article_texts[1000])

6495
['SpaceX Starship Blows Up Minutes After Launch', 'SpaceX’s Starship rocket, the most powerful ever built, blasted off on an unpiloted maiden flight Thursday, flying for more than two minutes before exploding. What do you think?', [('“Everything explodes eventually.”', 'Kat Alvarez, Apartment Stager'), ('“We’re on the doorstep of a bold new era of space disasters.”', 'Andy Hood, Jar Sealer'), ('“I’d like to see China blow up a rocket in less than two minutes.”', 'Oscar Huerta, PR Educator')]]
['Study Finds Plants Make Noises When Stressed', 'A new study has found that “stressed” plants that have not been watered for several days or had their stems cut emit ultrasonic clicking noises that are undetectable to the human ear but may be heard by insects and other animals. What do you think?', [('“Strange, mine just turn brown and die.”', 'Tracey Puentes, Monologue Editor'), ('“They should suffer in silence like the rest of us.”', 'Isaac McCafferty, Chief Consultant'), ('“Nothing a litt